In [4]:
import itertools

listing_id = input('what is the listing_id?:\t')

kp_mt = core_query('''
                    SELECT
                      phrase,
                      (CASE WHEN matchtype = 6
                        THEN 1
                       ELSE matchtype END) AS mt
                    FROM control.listing l
                      JOIN control.keyphrase k
                        ON l.keyphraseid = k.id
                    WHERE l.id = {pLISTINGID}
                '''.format(pLISTINGID = listing_id))

phrase, mt = kp_mt.loc[0, 'phrase'].split(), kp_mt.loc[0, 'mt']

tokens = algo_query('''
                    SELECT skt.id AS skt_id
                    FROM control.keyphrase_token kt
                      JOIN control.stemmed_keyphrase_token skt
                        ON kt.stemmed_keyphrase_token = skt.stemmed_keyphrase_token
                           AND kt.keyphrase_token IN {pKP}
                    '''.format(pKP = tuple(phrase))).skt_id.tolist()

features = tuple(itertools.product(tokens, [mt]))

coeffs = algo_query('''
                    SELECT sum(coefficient)
                    FROM control.quality_model_parameter
                    WHERE (component_0, component_1) IN {pFEATURES}
                     OR feature_id = 0
                    '''.format(pFEATURES = features))

(np.exp(coeffs) / (1 + np.exp(coeffs))).values[0][0]

what is the listing_id?:	1368496515


0.61249356947772726